In [238]:
import rasterio
from rasterio.plot import show
from matplotlib import pyplot
import plotly.io as pio
import plotly.express as px
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output

In [239]:
def visualize_GEOTIFF(layer_url):
    dataset = rasterio.open(layer_url)
    pyplot.figure(figsize=(50, 25))
    show(dataset)
    
def visualize_COG(layer_url):
    pio.renderers.default = 'notebook'
    # pio.renderers.default = 'jupyterlab'
    # pio.renderers.default = 'colab'
    
    # seed value, for mapbox to load in colab
    df = pd.DataFrame([[1001, 5.3],[1001, 5.3]])
    df.columns = ["flips", "unemp"]
    fig = px.choropleth_mapbox(
                                df, 
                                color='unemp',
                                color_continuous_scale="Viridis",
                                range_color=(0, 12),
                                mapbox_style="carto-positron",
                                zoom=4, center = {"lat": 33.543682, "lon": -86.779633},
                                opacity=0.5,
                              )
    #print(type(layer_url))
    fig.update_layout(
        mapbox_layers=[
            {
                "sourcetype": "raster",
                "source": [layer_url]
            }
          ])
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    #print(fig.layout.mapbox.layers)
    fig.update_layout(mapbox_style="stamen-terrain")
    fig.layout.mapbox.zoom = 1
    fig.show()

def drop_down(options, metadata={}, value=None):
    # Define the options for the dropdown
    dropdown_options = ['Option 1', 'Option 2', 'Option 3']

    # Create the dropdown widget
    dropdown = widgets.Dropdown(options=DATASETS, description='', value=value)

    # Define a function to handle the dropdown's value change
    def handle_dropdown_change(change):
        selected_option = change.new
        # Perform actions based on the selected option
        handle_dataset_input(metadata)

    # Attach the function to the dropdown's value change event
    dropdown.observe(handle_dropdown_change, names='value')

    # Display the dropdown widget
    display(dropdown)

def handle_dataset_input(metadata):
    if selected_dataset == 'TRMM_LIS_FULL':
        handle_trmm_lis_full(BASE_URL)
    elif selected_dataset == 'TRMM_LIS_SEASONAL':
        handle_trmm_lis_seasonal(BASE_URL)

def clear_cell(value):
    clear_output(wait=True)
    print("Select a dataset from the options below: ", end="")
    selected_dataset = drop_down(DATASETS, value=value)

In [240]:
#handler for different datasets
def handle_trmm_lis_full(BASE_URL, metadata):
    clear_cell(value="TRMM_LIS_FULL")
    visualize_COG(BASE_URL+"VHRFC/201301/LIS/{z}/{x}/{y}.png?colormap=terrain&stretch_range=[0.325,23.426]")
def handle_trmm_lis_seasonal(BASE_URL, metadata):
    clear_cell(value="TRMM_LIS_SEASONAL")
    print("im in seasonal")
def handle_trmm_lis_monthly():
    pass
def handle_trmm_lis_diurnal():
    pass
def handle_trmm_lis_daily():
    pass
def handle_otd_full():
    pass
def handle_otd_monthly():
    pass
def handle_otd_diurnal():
    pass
def handle_otd_daily():
    pass
def handle_isslis():
    pass
def handle_hs3():
    pass
def handle_nalma():
    pass

In [241]:
#constants
DATASETS = ['TRMM_LIS_FULL','TRMM_LIS_SEASONAL','TRMM_LIS_MONTHLY', 'TRMM_LIS_DIURNAL', 'TRMM_LIS_DAILY']
BASE_URL = 'https://wug8w3fg42.execute-api.us-west-2.amazonaws.com/development/singleband/'

In [242]:
print("Select a dataset from the options below: ", end="")
selected_dataset = drop_down(DATASETS)

Select a dataset from the options below: 

Dropdown(options=('TRMM_LIS_FULL', 'TRMM_LIS_SEASONAL', 'TRMM_LIS_MONTHLY', 'TRMM_LIS_DIURNAL', 'TRMM_LIS_DAIL…